In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# importo modulos de pytorch

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
import torch

### Descargo archivos y hago data frame

In [ ]:
# Importar librerías necesarias
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split

# Montar Google Drive (si estás trabajando en Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Definir rutas a las canciones de los artistas
import os
ruta_tini = '/content/drive/MyDrive/NLP/Canciones/TINI'
ruta_spinetta = '/content/drive/MyDrive/NLP/Canciones/Spinetta'
ruta_callejero = '/content/drive/MyDrive/NLP/Canciones/Callejero'
ruta_dillom = '/content/drive/MyDrive/NLP/Canciones/Dillom'

artistas = ['tini', 'spinetta', 'callejero', 'dillom']

# Cargar las canciones
def cargar_canciones(ruta):
    canciones = []
    archivos = os.listdir(ruta)
    for archivo in archivos:
        ruta_archivo = os.path.join(ruta, archivo)
        with open(ruta_archivo, 'r') as archivo_txt:
            contenido = archivo_txt.read()
            canciones.append(contenido)
    return canciones

tini = cargar_canciones(ruta_tini)
spinetta = cargar_canciones(ruta_spinetta)
callejero = cargar_canciones(ruta_callejero)
dillom = cargar_canciones(ruta_dillom)

# Preparar el DataFrame
data = {'cancion': [], 'artista': []}

for artista, canciones in zip(artistas, [tini, spinetta, callejero, dillom]):
    for cancion in canciones:
        data['cancion'].append(cancion)
        data['artista'].append(artista)

df = pd.DataFrame(data)
df


Mounted at /content/drive


,cancion,artista
0,Recuerdo que contigo fue tan diferente\nTan na...,tini
1,"(Vos tenés ganas de enamorarte)\n(Pero, despué...",tini
2,"Hay un sueño que late\nDentro de ti, dentro de...",tini
3,Sé que por un tiempo voy a navegar sin rumbo\n...,tini
4,Te prometo no llorar\nEsperarte y no olvidar\n...,tini
...,...,...
119,"[Jim Jone]\n(Death is)\n(I tell you, I don't c...",dillom
120,Déjenme solo\nEl mundo está en mi contra\nNo s...,dillom
121,"Dios mío, dame mi sueño de paz\nY no de pastil...",dillom
122,Durante aquella última noche de campamento\nLa...,dillom


In [ ]:
model_name = "bert-base-uncased"


In [ ]:
model_name = "distilbert-base-uncased"  # or "roberta-base"


###Modelo

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
X_train, y_train = train_df['cancion'].tolist(), train_df['artista'].tolist()
X_test, y_test = test_df['cancion'].tolist(), test_df['artista'].tolist()

# Codificar etiquetas
label_map = {label: idx for idx, label in enumerate(artistas)}
y_train = [label_map[label] for label in y_train]
y_test = [label_map[label] for label in y_test]

# Crear datasets de Hugging Face
train_dataset = Dataset.from_dict({'text': X_train, 'label': y_train})
test_dataset = Dataset.from_dict({'text': X_test, 'label': y_test})

# Cargar el tokenizador y el modelo BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(artistas))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenizar los datos
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Agrupar los datos con relleno dinámico
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Esto de aca es porque si no hay que usar una api para ir viendo algo de la loss o no se y como se va actualizando que es un quilombo entonces esto hace que sea caja negra no pasa nada

In [ ]:
import os

# Desactiva Weights & Biases
os.environ["WANDB_MODE"] = "disabled"

# Luego, continúa con tu código como de costumbre


In [ ]:
import wandb
wandb.init(mode="disabled")


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
import random
import numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For GPUs
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False   # Avoid randomness in certain operations

# Set the seed before starting training
set_seed(42)

In [ ]:
# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    num_train_epochs=25,
    weight_decay=0.009, #probe con 0.001 y daba re mal, con 0.01 y 0.1 daba parecido pero mas grande mas loss
    seed=42,  # Set the seed for reproducibility
    logging_dir='./logs',
    gradient_accumulation_steps=8,  # Si el batch size es pequeño, acumula gradientes para simular un batch más grande
    lr_scheduler_type="cosine_with_restarts",  # Add this line for cosine decay with warmup
    warmup_ratio=0.1,  # Fraction of steps for warmup


)

import evaluate

# Cargar la métrica de accuracy usando evaluate
accuracy_metric = evaluate.load("accuracy")

# Definir una función para calcular la métrica de accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)

    # Calculate metrics for multiclass classification
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')  # Use 'weighted', 'macro', or 'micro'
    recall = recall_score(labels, preds, average='weighted')        # Use 'weighted', 'macro', or 'micro'
    f1 = f1_score(labels, preds, average='weighted')                # Use 'weighted', 'macro', or 'micro'

    # Confusion matrix for multiclass
    cm = confusion_matrix(labels, preds)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "confusion_matrix": cm.tolist(),  # Return as a list for JSON compatibility if needed
    }

from transformers import get_scheduler
from transformers import AdamW  # or from torch.optim import AdamW


optimizer = AdamW(model.parameters(), lr=3e-5)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_dataset)
)


from transformers import TrainerCallback
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Custom callback to integrate ReduceLROnPlateau with Trainer
class CustomCallback(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        self.scheduler = ReduceLROnPlateau(kwargs['optimizer'], mode='min', factor=0.5, patience=2, min_lr=1e-7)

    def on_epoch_end(self, args, state, control, **kwargs):
        # Check if log_history has entries
        if len(state.log_history) > 0:
            validation_loss = state.log_history[-1].get("eval_loss")
            if validation_loss is not None:
                self.scheduler.step(validation_loss)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # Pasar la función de métrica
    optimizers=(optimizer, lr_scheduler),  # Include optimizer and scheduler
    callbacks=[CustomCallback]

)

# Entrenar el modelo
trainer.train()

# Evaluar el modelo
eval_results = trainer.evaluate()
print(f"Resultados de evaluación: {eval_results}")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-13-48ceb107c3ac>:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Confusion Matrix
0,No log,1.370752,0.440000,0.257778,0.440000,0.309000,"[[0, 1, 7, 0], [0, 7, 0, 0], [0, 0, 4, 0], [0, 1, 5, 0]]"
1,No log,1.328518,0.560000,0.556364,0.560000,0.477657,"[[3, 2, 3, 0], [0, 7, 0, 0], [0, 0, 4, 0], [0, 2, 4, 0]]"
2,No log,1.252180,0.720000,0.584762,0.720000,0.630825,"[[8, 0, 0, 0], [1, 6, 0, 0], [0, 0, 4, 0], [3, 0, 3, 0]]"
4,No log,1.048576,0.680000,0.720727,0.680000,0.671043,"[[5, 2, 0, 1], [0, 6, 0, 1], [0, 0, 4, 0], [0, 3, 1, 2]]"
5,No log,0.947843,0.760000,0.786182,0.760000,0.740952,"[[6, 1, 0, 1], [0, 7, 0, 0], [0, 0, 4, 0], [0, 3, 1, 2]]"
6,No log,0.880490,0.800000,0.866182,0.800000,0.778667,"[[7, 1, 0, 0], [0, 7, 0, 0], [0, 0, 4, 0], [0, 3, 1, 2]]"
8,No log,0.681841,0.800000,0.793000,0.800000,0.787556,"[[6, 0, 1, 1], [0, 7, 0, 0], [0, 0, 4, 0], [2, 1, 0, 3]]"
9,No log,0.711748,0.840000,0.898182,0.840000,0.836444,"[[7, 1, 0, 0], [0, 7, 0, 0], [0, 0, 4, 0], [0, 3, 0, 3]]"
10,No log,0.630709,0.840000,0.876000,0.840000,0.830588,"[[7, 1, 0, 0], [0, 7, 0, 0], [0, 0, 4, 0], [1, 2, 0, 3]]"
12,No log,0.601546,0.840000,0.876000,0.840000,0.830588,"[[7, 1, 0, 0], [0, 7, 0, 0], [0, 0, 4, 0], [1, 2, 0, 3]]"


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Trainer is attempting to log a value of "[[0, 1, 7, 0], [0, 7, 0, 0], [0, 0, 4, 0], [0, 1, 5, 0]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Trainer is attempting to log a value of "[[3, 2, 3, 0], [0, 7, 0, 0], [0, 0, 4, 0], [0, 2, 4, 0]]" of t

Trainer is attempting to log a value of "[[8, 0, 0, 0], [0, 7, 0, 0], [0, 0, 4, 0], [0, 2, 0, 4]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Resultados de evaluación: {'eval_loss': 0.5153569579124451, 'eval_accuracy': 0.92, 'eval_precision': 0.9377777777777777, 'eval_recall': 0.92, 'eval_f1': 0.917, 'eval_confusion_matrix': [[8, 0, 0, 0], [0, 7, 0, 0], [0, 0, 4, 0], [0, 2, 0, 4]], 'eval_runtime': 0.3934, 'eval_samples_per_second': 63.549, 'eval_steps_per_second': 17.794, 'epoch': 24.0}


Algunos tensores están en el dispositivo cuda (GPU), mientras que otros están en cpu (CPU). Para resolverlo, asegúrate de mover todos los tensores al mismo dispositivo. Vamos a ajustar la función obtener_embeddings para que se ejecute en el dispositivo correcto.
ni idea pero yo confio

In [ ]:
# Detectar el dispositivo (gpu si está disponible)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Mover el modelo al dispositivo

# Obtener embeddings asegurándonos de que las entradas estén en el mismo dispositivo
def obtener_embeddings(canciones):
    inputs = tokenizer(canciones, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Mover entradas al dispositivo
    outputs = model(**inputs)
    embeddings = outputs.logits  # Embeddings de la última capa antes de la clasificación
    return embeddings

# Ejemplo de cómo obtener los embeddings
embeddings_train = obtener_embeddings(X_train[:5])
print("Embeddings de ejemplo para las primeras canciones de entrenamiento:", embeddings_train)


Embeddings de ejemplo para las primeras canciones de entrenamiento: tensor([[ 0.0199, -1.3870,  1.7307, -0.8279],
        [ 1.7339, -0.4184, -0.8613, -0.6945],
        [ 1.3620, -0.1352, -1.0893, -0.3487],
        [-0.3267,  2.1932, -1.0014, -0.9514],
        [-0.2774, -1.2081, -0.6981,  2.0446]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


Para las Nereas, le pedí a GPT que me haga algo más user friendly, tipo que le doy una canción y que me la clasifique... quiero leer output lindos no ACCURACY 0.86.

In [ ]:
# Número de clases en el modelo
num_clases = model.config.num_labels
print(f"El modelo tiene {num_clases} clases.")

import torch
import torch.nn.functional as F  # Importar softmax

def predecir_artista(cancion, tokenizer, model, label_map):
    # Detectar el dispositivo en el que se encuentra el modelo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Asegurarse de que el modelo esté en el dispositivo

    # Tokenizar la entrada y moverla al mismo dispositivo
    inputs = tokenizer(cancion, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Mover entradas al dispositivo

    # Pasar la entrada por el modelo y obtener las predicciones
    model.eval()  # Configurar el modelo en modo de evaluación
    with torch.no_grad():  # No calcular gradientes para una inferencia más rápida
        outputs = model(**inputs)
        logits = outputs.logits

    # Aplicar softmax a los logits para obtener probabilidades
    probs = F.softmax(logits, dim=-1)

    # Obtener la predicción de la clase (índice con la mayor probabilidad)
    pred_idx = probs.argmax(dim=-1).item()

    # Obtener las probabilidades de cada clase
    probabilidades = probs.squeeze().cpu().numpy()

    # Crear un diccionario de probabilidades con las clases
    probabilidades_clase = {label_map[i]: probabilidad for i, probabilidad in enumerate(probabilidades)}

    # Validar si el índice está en el rango de las clases posibles
    if pred_idx in label_map:
        artista_predicho = label_map[pred_idx]
    else:
        artista_predicho = "Desconocido"  # Si el índice no es válido

    return artista_predicho, probabilidades_clase



El modelo tiene 4 clases.


In [ ]:
label_map = {0: 'tini', 1: 'spinetta', 2: 'callejero', 3: 'dillom'}  # Ajusta según sea necesario


Me devuelve la probabilidad de que sea de cada artista

In [ ]:
while True:
    print("\nIntroduce la letra de una canción (o escribe 'salir' para terminar):")
    cancion_input = input("> ")
    if cancion_input.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Llamar a la función para predecir el artista
    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)
    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")


Introduce la letra de una canción (o escribe 'salir' para terminar):
>  visualizaciones de letras 43.851  IConic.mp3 Emilia Letra Significado  Traducciones  (Somo' icóni-icóni-icóni-icóni-icóni-icónico') (Somo' icóni-icóni-icóni-icóni-icóni-icónico')  En la calle, la gente cree que te conoce, pero solo yo sé Lo que sientes tú debajo 'el tattoo después de las doce Se prenden las luce', Prada te luce y yo estoy en pose El mundo nos mira, to's nos conocen  Porque somo' icónico' Con solo un beso me desnuda en automático Hacemos historia, pero no por storie' Me mata sin forty, tú y yo somo' iconic, eh, eh  Porque, porque somo' icónico' Con solo un beso me desnuda en automático Hacеmos historia, pero no por storie' Me mata sin forty, tú y yo somo' iconic, еh, eh  Yo-yo-yo sé que estoy bien, pero tú me completas La otra línea de mi camiseta Vidrio negro como tu chaqueta Teñido de rojo pa' que no se metan  Yo sé Que la distancia a veces nadie la ve Mi piel te siente y mis ojo' no te ven No sé

#AHORA LO GUARDAMOS

In [ ]:
import os

# Imprimir la ubicación actual en Google Drive
current_drive_path = '/content/drive/My Drive/'
os.chdir(current_drive_path)  # Cambiar al directorio base del Drive si es necesario
print(f"Directorio actual: {os.getcwd()}")


import json

# Ruta en Google Drive (directorio actual)
ruta_guardado = os.getcwd() + '/modelo_guardado_entrenamiento31'

# Guardar el modelo y el tokenizer
model.save_pretrained(ruta_guardado)
tokenizer.save_pretrained(ruta_guardado)

# Guardar el label_map
label_map_path = f"{ruta_guardado}/label_map.json"
with open(label_map_path, 'w') as f:
    json.dump(label_map, f)

print(f"Modelo y metadatos guardados en {ruta_guardado}")



Directorio actual: /content/drive/My Drive
Modelo y metadatos guardados en /content/drive/My Drive/modelo_guardado_entrenamiento31


#AHORA LO IMPORTAMOS ASUMIMOS QUE YA SE GUARDO

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
label_map = {0: 'tini', 1: 'spinetta', 2: 'callejero', 3: 'dillom'}  # Ajusta según sea necesario


In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Ruta al modelo guardado
ruta_local = '/content/drive/My Drive/modelo_guardado_entrenamiento31'

# Cargar el modelo
model = AutoModelForSequenceClassification.from_pretrained(ruta_local)

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(ruta_local)

print("Modelo y tokenizer cargados correctamente.")


Modelo y tokenizer cargados correctamente.


In [5]:
# Número de clases en el modelo
num_clases = model.config.num_labels
print(f"El modelo tiene {num_clases} clases.")

import torch
import torch.nn.functional as F  # Importar softmax

def predecir_artista(cancion, tokenizer, model, label_map):
    # Detectar el dispositivo en el que se encuentra el modelo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Asegurarse de que el modelo esté en el dispositivo

    # Tokenizar la entrada y moverla al mismo dispositivo
    inputs = tokenizer(cancion, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Mover entradas al dispositivo

    # Pasar la entrada por el modelo y obtener las predicciones
    model.eval()  # Configurar el modelo en modo de evaluación
    with torch.no_grad():  # No calcular gradientes para una inferencia más rápida
        outputs = model(**inputs)
        logits = outputs.logits

    # Aplicar softmax a los logits para obtener probabilidades
    probs = F.softmax(logits, dim=-1)

    # Obtener la predicción de la clase (índice con la mayor probabilidad)
    pred_idx = probs.argmax(dim=-1).item()

    # Obtener las probabilidades de cada clase
    probabilidades = probs.squeeze().cpu().numpy()

    # Crear un diccionario de probabilidades con las clases
    probabilidades_clase = {label_map[i]: probabilidad for i, probabilidad in enumerate(probabilidades)}

    # Validar si el índice está en el rango de las clases posibles
    if pred_idx in label_map:
        artista_predicho = label_map[pred_idx]
    else:
        artista_predicho = "Desconocido"  # Si el índice no es válido

    return artista_predicho, probabilidades_clase

El modelo tiene 4 clases.


In [ ]:
while True:
    print("\nIntroduce la letra de una canción (o escribe 'salir' para terminar):")
    cancion_input = input("> ")
    if cancion_input.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Llamar a la función para predecir el artista
    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)
    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")